# Capstone Project Notebook
This notebook will be used to fulfill the capstone project on Coursera.

In [37]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

## Getting the Postal Codes and neighborhoods of Toronto

Fetch the table from the wikipedia page with the manually provided link:

In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
soup = BeautifulSoup(website_url,"lxml")

In [4]:
my_table = soup.find("table",{"class":"wikitable sortable"})

In [5]:
postal_codes = []
boroughs = []
neighbourhoods = []
for row in my_table.find_all('tr')[1:]:
    postal_codes_cell = row.find_all('td')[0]
    postal_codes.append(postal_codes_cell.text)
for row in my_table.find_all('tr')[1:]:
    boroughs_cell = row.find_all('td')[1]
    boroughs.append(boroughs_cell.text)
for row in my_table.find_all('tr')[1:]:
    neighbourhoods_cell = row.find_all('td')[2]
    neighbourhoods.append(neighbourhoods_cell.text)

This is then converted to a dataframe

In [6]:
dictionary = {"Postal Code": postal_codes, "Boroughs": boroughs, "Neighborhood": neighbourhoods}
df = pd.DataFrame(dictionary)
df = df.apply(lambda x: x.str.rstrip("\n"))

Rows without boroughs are dropped from the dataframe.

In [7]:
df = df.replace("Not assigned", np.NaN)
df.dropna(subset=["Boroughs"], inplace=True)

Rows without neighbourhoods are filled with the borough as neighbourhood.

In [9]:
df.loc[df["Neighborhood"]!=df["Neighborhood"], "Neighborhood"] = df["Boroughs"]

In [10]:
df.shape

(103, 3)

## Getting coordinates

In [11]:
def do_not_run():
    import geocoder # import geocoder

    # initialize your variable to None
    lat_lng_coords = None

    for postal in df["Postal Code"]:

        # loop until you get the coordinates
        while(lat_lng_coords is None):
          g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
          lat_lng_coords = g.latlng

        df.loc[df["Postal Code"]==postal, "Latitude"] = lat_lng_coords[0]
        df.loc[df["Postal Code"]==postal, "Longitude"] = lat_lng_coords[1]

Since the geocoder never finished compiling, I'm importing the source manually from the csv.

In [12]:
coordinate_s = pd.read_csv("https://cocl.us/Geospatial_data")

In [13]:
df = pd.merge(df, coordinate_s, on="Postal Code", how="left")

In [14]:
df.head()

,Postal Code,Boroughs,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Explore the neighborhoods of Torono

First we'll reduce the boroughs to the ones containing "Toronto" in their name.

In [15]:
df_red = df[df["Boroughs"].str.contains("Toronto")]

We'll need to set some parameters for the foursquare API.

In [16]:
CLIENT_ID = "SBLODBLCGZNT4MIP04BPKDRDIMQETLUIWVMD0H5KAH5OHPWS"
CLIENT_SECRET = "0BF4FQMSHJ5BASM1USWS0FARALO04VMN2WM34WOGXKYJ5D3G"
version = "20200110"
base_url = "https://api.foursquare.com/v2/venues/search?"
LIMIT = 100
radius = 500

Using the function from the lab to fetch venues for given neighborhoods.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=df_red["Neighborhood"],
                                  latitudes=df_red["Latitude"],
                                  longitudes=df_red["Longitude"]
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


Let's see the size of the output we received.

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(1614, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [21]:
# The venues per neighborhood
toronto_venues.groupby('Neighborhood')["Venue"].count().sort_values(ascending=False)

Neighborhood
Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Garden District, Ryerson                                                                                      100
Harbourfront East, Union Station, Toronto Islands                                                             100
Toronto Dominion Centre, Design Exchange                                                                      100
Stn A PO Boxes                                                                                                 98
Richmond, Adelaide, King                                                                                       96
St. James Town                                                                                                 80
Church and Wellesley                                                       

In [22]:
# Analyze the category numbers
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


Let's analyze the number of venues per neighborhood.
Adding type of venues as columns to group afterwards.

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.00000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.214286,0.142857,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000
4,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.016129,0.00000,0.000000,0.016129
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000
6,Church and Wellesley,0.025974,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,...,0.012987,0.012987,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.012987,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.00000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.028571,0.00000,0.00,0.000000,0.00000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues



In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4             Bar  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2      Recording Studio  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.21
1  Airport Terminal  0.14
2             Plane  0.07
3           Airport  0.07
4          Boutique  0.07


----Central Bay Street----
                venue  freq
0       

#### Let's put that into a _pandas_ dataframe
First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Restaurant,Farmers Market,Cheese Shop,Bakery,Grocery Store,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Fast Food Restaurant,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Boutique,Sculpture Garden,Rental Car Location,Plane,Boat or Ferry,Harbor / Marina,Airport Lounge,Airport Food Court
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,Juice Bar,Korean Restaurant


## Cluster Neighborhoods
Run _k_-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_red

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Boroughs,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Breakfast Spot,Restaurant,Theater,Yoga Studio,Shoe Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Beer Bar,Falafel Restaurant,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Café,Fast Food Restaurant,Theater
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Farmers Market,Creperie,Cosmetics Shop,Gym
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wine Bar,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine the clusters:

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Café,Bakery,Pub,Park,Breakfast Spot,Restaurant,Theater,Yoga Studio,Shoe Store
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Beer Bar,Falafel Restaurant,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Café,Fast Food Restaurant,Theater
15,Downtown Toronto,0,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Farmers Market,Creperie,Cosmetics Shop,Gym
19,East Toronto,0,Trail,Health Food Store,Pub,Wine Bar,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Restaurant,Farmers Market,Cheese Shop,Bakery,Grocery Store,Museum
24,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,Juice Bar,Korean Restaurant
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,Gym,Thai Restaurant,Deli / Bodega,Bookstore,Pizza Place,Cosmetics Shop
31,West Toronto,0,Bakery,Pharmacy,Grocery Store,Middle Eastern Restaurant,Music Venue,Park,Pizza Place,Coffee Shop,Pool,Café
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Scenic Lookout,Restaurant,Brewery,Italian Restaurant,Sporting Goods Shop


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Wine Bar,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,2,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Coffee Shop,Candy Store


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,3,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Wine Bar,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,4,Park,Playground,Restaurant,Wine Bar,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
91,Downtown Toronto,4,Park,Playground,Trail,Wine Bar,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
